### Tutorial links: 

[Web Scraping with Pandas](https://www.youtube.com/watch?v=oF-EMiPZQGA)

[【 台灣股市資訊網】Post爬蟲大公開](https://medium.com/pythonstock/%E5%8F%B0%E7%81%A3%E8%82%A1%E5%B8%82%E8%B3%87%E8%A8%8A%E7%B6%B2-post%E7%88%AC%E8%9F%B2%E5%A4%A7%E5%85%AC%E9%96%8B-%E9%99%84-python%E7%A8%8B%E5%BC%8F%E7%A2%BC-e296238f9ef4)

[Sending "User-agent" using Requests library in Python](https://stackoverflow.com/questions/10606133/sending-user-agent-using-requests-library-in-python)


### Goal

To scrape 2 webpages and save the combine the data into excel

- [券商進出排行](https://5850web.moneydj.com/z/zg/zgb/zgb0.djhtm?a=1470&b=1470&c=B&d=1)

- [Goodinfo!](https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9+%28%E9%AB%98%E2%86%92%E4%BD%8E%29%40%40%E6%88%90%E4%BA%A4%E5%83%B9%40%40%E7%94%B1%E9%AB%98%E2%86%92%E4%BD%8E)

![Optional Text](IMG_4189.JPG)

### Get 2 tables from [券商進出排行榜](https://5850web.moneydj.com/z/zg/zgb/zgb0.djhtm?a=1470&b=1470&c=B&d=1)

Use Pandas scraper to get tables

In [ ]:
import requests
import pandas as pd
import re
import datetime
import time

url = "https://5850web.moneydj.com/z/zg/zgb/zgb0.djhtm?a=1470&b=1470&c=B&d=1"
response = requests.get(url)
response.encoding = 'big5'  # Specify the correct encoding here

scraper = pd.read_html(response.text)



In [ ]:
# Seel all the tables available and find the one you want

# for index, table in enumerate(scraper):
#     print("***********************************************************")
#     print("Index", index)
#     print(table)


In [ ]:
# use rex to make "<!-- \tGenLink2stk('AS2891','中信金'); //-->" to "2891中信金"
def get_stock_id_name(text):
    m = re.search(r"GenLink2stk\('AS(\d+)','(.+?)'\);", text)
    if m:
        return m.group(1) + m.group(2)
    else:
        return text

In [ ]:
# use rex to add "買超" to the beginning of the column name
def add_buy(text):
    return "買超-" + text

# make the above code a function with a parameter of dataframe
def process_table(df):
    # apply to the first column of table
    stock_id_name = df.iloc[:, 0].apply(get_stock_id_name)
    # put it back to the table
    df.iloc[:, 0] = stock_id_name
    # remove the first row
    df = df.iloc[1:] 
    # use the first row as header
    df.columns = df.iloc[0]
    # remove the first row again
    df = df.iloc[1:]
    # add "買超-/賣超-" to the beginning of the column name
    df.columns = df.columns.map(add_buy)
    return df

In [ ]:
df_buy = process_table(scraper[3])

# df_sell = process_table(scraper[4])

In [ ]:
# edit the column of "買超-券商名稱". make it separate digits and letters. both of them are strings
def separate_digits_letters(text):
    m = re.search(r"(\d+)([a-zA-Z]+)", text)
    if m:
        return m.group(1), m.group(2)
    else:
        return text, ""
    
df_buy["買超-券商代號"], df_buy["買超-券商名稱"] = zip(*df_buy["買超-券商名稱"].map(separate_digits_letters))

# make "買超-券商代號" the first column
df_buy = df_buy[["買超-券商代號", "買超-券商名稱"] + [col for col in df_buy.columns if col not in ["買超-券商代號", "買超-券商名稱"]]]


### Get today and yeaterday's all stocks data from [Goodinfo](https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9+%28%E9%AB%98%E2%86%92%E4%BD%8E%29%40%40%E6%88%90%E4%BA%A4%E5%83%B9%40%40%E7%94%B1%E9%AB%98%E2%86%92%E4%BD%8E)

- POST method

- Query parameters
    - MARKET_CAT
    - INDUSTRY_CAT
    - RPT_TIME
    - RANK


In [ ]:
# get yesterday's date
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday = yesterday.strftime("%Y/%m/%d")


In [ ]:
# Get all stocks data (today)
# Initialize the final dataframe
all_stocks = pd.DataFrame()

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
}

for rank in range(0, 8): # (0, 8)
    url = f"https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9+%28%E9%AB%98%E2%86%92%E4%BD%8E%29%40%40%E6%88%90%E4%BA%A4%E5%83%B9%40%40%E7%94%B1%E9%AB%98%E2%86%92%E4%BD%8E&RANK={rank}"
    
    response = requests.post(url, headers=headers)
    response.encoding = 'uft8'  # Specify the correct encoding here
    
    scraper = pd.read_html(response.text)
    
    stocks = scraper[61]
    
    all_stocks = pd.concat([all_stocks, stocks], axis=0)

    print(f"Fetched {rank} page for all_stocks")
    
    time.sleep(10)
    
    
# delete the rows where "名稱" == "名稱"
all_stocks = all_stocks[all_stocks["名稱"] != "名稱"]    

# delete the columns which have all NaN
all_stocks = all_stocks.dropna(axis=1, how="all")

    

In [ ]:
# Get all stocks yesterday

# get yesterday's date
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday = yesterday.strftime("%Y/%m/%d") # e.g. '2024/03/22'

# Initialize the final dataframe
all_stocks_yesterday = pd.DataFrame()

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
}

for rank in range(0, 8): # (0, 8) -> 2m16s
    url = f"https://goodinfo.tw/tw2/StockList.asp?MARKET_CAT=%E7%86%B1%E9%96%80%E6%8E%92%E8%A1%8C&INDUSTRY_CAT=%E6%88%90%E4%BA%A4%E5%83%B9+%28%E9%AB%98%E2%86%92%E4%BD%8E%29%40%40%E6%88%90%E4%BA%A4%E5%83%B9%40%40%E7%94%B1%E9%AB%98%E2%86%92%E4%BD%8E&RANK={rank}&RPT_TIME={yesterday}"
    
    response = requests.post(url, headers=headers)
    response.encoding = 'uft8'  # Specify the correct encoding here
    
    scraper = pd.read_html(response.text)
    
    stocks = scraper[61]
    
    all_stocks_yesterday = pd.concat([all_stocks, stocks], axis=0)

    print(f"Fetched {rank} page for all_stocks_yesterday")
    
    time.sleep(10)
    
    
# delete the rows where "名稱" == "名稱"
all_stocks_yesterday = all_stocks[all_stocks["名稱"] != "名稱"]    

# delete the columns which have all NaN
all_stocks_yesterday = all_stocks.dropna(axis=1, how="all")


In [ ]:
# keep only "代號", "成交 張數" and  "成交額 (百萬)" in all_stocks_yesterday
all_stocks_yesterday = all_stocks_yesterday[["代號", "成交 張數", "成交額 (百萬)"]]
all_stocks_yesterday.head()

In [ ]:
# rename "成交 張數" and  "成交額 (百萬)" to "前一日成交 張數" and  "前一日成交額 (百萬)"
all_stocks_yesterday.columns = ["代號", "前一日成交 張數", "前一日成交額 (百萬)"]
all_stocks_yesterday.head()

In [ ]:
# merge all_stocks and all_stocks_yesterday based on "代號"
all_stocks = all_stocks.merge(all_stocks_yesterday, on="代號", how="left")
all_stocks.head()

In [ ]:
# merge df_buy and all_stocks based on "買超-券商代號" and "代號"
df_buy = df_buy.merge(all_stocks, left_on="買超-券商代號", right_on="代號", how="left")
df_buy.head()

In [ ]:
# save df_buy to excel file in a sheet named "買超"
df_buy.to_excel("券商進出排行_draft.xlsx", sheet_name="買超", index=False)